In [1]:
import pandas as pd
# If dataset.csv is in the same folder as your script
movies = pd.read_csv('dataset.csv')

print(movies)

          id                                          title  \
0        278                       The Shawshank Redemption   
1      19404                    Dilwale Dulhania Le Jayenge   
2        238                                  The Godfather   
3        424                               Schindler's List   
4        240                         The Godfather: Part II   
...      ...                                            ...   
9995   10196                             The Last Airbender   
9996  331446                       Sharknado 3: Oh Hell No!   
9997   13995                                Captain America   
9998    2312  In the Name of the King: A Dungeon Siege Tale   
9999  455957                                         Domino   

                                                 genre original_language  \
0                                          Drama,Crime                en   
1                                 Comedy,Drama,Romance                hi   
2              

In [2]:
movies.describe()


,id,popularity,vote_average,vote_count
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,161243.505000,34.697267,6.621150,1547.309400
std,211422.046043,211.684175,0.766231,2648.295789
min,5.000000,0.600000,4.600000,200.000000
25%,10127.750000,9.154750,6.100000,315.000000
50%,30002.500000,13.637500,6.600000,583.500000
75%,310133.500000,25.651250,7.200000,1460.000000
max,934761.000000,10436.917000,8.700000,31917.000000


In [3]:
movies.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [5]:
movies = movies[['id', 'title', 'genre', 'overview']]

movies.columns

Index(['id', 'title', 'genre', 'overview'], dtype='object')

In [6]:
movies['tags'] = movies['overview'] + movies['genre']

In [6]:
#new_data = movies.drop(columns= ['overview', 'genre'])

In [8]:
new_data = movies.head(5000)

In [9]:
new_data

,id,title,genre,overview,tags
0,278,The Shawshank Redemption,"Drama,Crime",Framed in the 1940s for the double murder of h...,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance","Raj is a rich, carefree, happy-go-lucky second...","Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Drama,Crime","Spanning the years 1945 to 1955, a chronicle o...","Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,"Drama,History,War",The true story of how businessman Oskar Schind...,The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,"Drama,Crime",In the continuing saga of the Corleone crime f...,In the continuing saga of the Corleone crime f...
...,...,...,...,...,...
4995,32042,Night Moves,"Thriller,Crime,Mystery",Private detective and former football player H...,Private detective and former football player H...
4996,11422,Midway,"Action,Drama,History,War",This war drama depicts the U.S. and Japanese f...,This war drama depicts the U.S. and Japanese f...
4997,11412,The Long Kiss Goodnight,"Crime,Action,Mystery,Thriller","Samantha Caine, suburban homemaker, is the ide...","Samantha Caine, suburban homemaker, is the ide..."
4998,9614,Happy Gilmore,Comedy,Failed hockey player-turned-golf whiz Happy Gi...,Failed hockey player-turned-golf whiz Happy Gi...


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
cv = CountVectorizer(max_features = 8000, stop_words = 'english')

In [12]:
vector = cv.fit_transform(new_data['tags'].values.astype('U')).toarray()

In [13]:
vector.shape

(5000, 8000)

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
similarity = cosine_similarity(vector)

In [16]:
similarity

array([[1.        , 0.05634362, 0.12888482, ..., 0.09416472, 0.        ,
        0.16035675],
       [0.05634362, 1.        , 0.07624929, ..., 0.06189845, 0.09284767,
        0.05270463],
       [0.12888482, 0.07624929, 1.        , ..., 0.11327309, 0.        ,
        0.19289713],
       ...,
       [0.09416472, 0.06189845, 0.11327309, ..., 1.        , 0.        ,
        0.05872202],
       [0.        , 0.09284767, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.16035675, 0.05270463, 0.19289713, ..., 0.05872202, 0.        ,
        1.        ]])

In [17]:
similarity.shape

(5000, 5000)

In [18]:
distance = sorted(list(enumerate(similarity[2])), reverse = True, key = lambda vector:vector[1])

for i in distance[0:5]:
    print(new_data.iloc[i[0]].title)

The Godfather
The Godfather: Part II
Joker
Bomb City
Felon


In [19]:
def recommand(movies):
    index = new_data[new_data['title'] == movies].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse = True, key = lambda vector:vector[1])
    for i in distance[0:5]:
        print(new_data.iloc[i[0]].title)

In [76]:
recommand("Iron Man")

Iron Man
Iron Man 3
Guardians of the Galaxy Vol. 2
Star Wars: Episode III - Revenge of the Sith
Avengers: Age of Ultron


In [20]:
import pickle
import gzip

In [21]:
with gzip.open('similarity.pkl.gz', 'wb') as f:
    pickle.dump(similarity, f)

In [24]:
pickle.dump(new_data, open("movies_list.pkl" , 'wb'))